# Python pandas

In [1]:
# Import statements go here

import pandas as pd
import statsmodels.api as sm

## Импорт и создание DataFrame 

Выполните код ниже, чтобы загрузить из файла `anes96.csv` данные [Американских национальных исследований выборов 1996 года](https://www.statsmodels.org/dev/datasets/generated/anes96.html).

In [2]:
df = pd.read_csv('anes96.csv')

Информация о содержимом датасета:

```plain
    Variables name definitions::

            popul - Census place population in 1000s
            TVnews - Number of times per week that respondent watches TV news.
            PID - Party identification of respondent.
                0 - Strong Democrat
                1 - Weak Democrat
                2 - Independent-Democrat
                3 - Independent-Indpendent
                4 - Independent-Republican
                5 - Weak Republican
                6 - Strong Republican
            age : Age of respondent.
            educ - Education level of respondent
                1 - 1-8 grades
                2 - Some high school
                3 - High school graduate
                4 - Some college
                5 - College degree
                6 - Master's degree
                7 - PhD
            income - Income of household
                1  - None or less than $2,999
                2  - $3,000-$4,999
                3  - $5,000-$6,999
                4  - $7,000-$8,999
                5  - $9,000-$9,999
                6  - $10,000-$10,999
                7  - $11,000-$11,999
                8  - $12,000-$12,999
                9  - $13,000-$13,999
                10 - $14,000-$14.999
                11 - $15,000-$16,999
                12 - $17,000-$19,999
                13 - $20,000-$21,999
                14 - $22,000-$24,999
                15 - $25,000-$29,999
                16 - $30,000-$34,999
                17 - $35,000-$39,999
                18 - $40,000-$44,999
                19 - $45,000-$49,999
                20 - $50,000-$59,999
                21 - $60,000-$74,999
                22 - $75,000-89,999
                23 - $90,000-$104,999
                24 - $105,000 and over
            vote - Expected vote
                0 - Clinton
                1 - Dole
            The following 3 variables all take the values:
                1 - Extremely liberal
                2 - Liberal
                3 - Slightly liberal
                4 - Moderate
                5 - Slightly conservative
                6 - Conservative
                7 - Extremely Conservative
            selfLR - Respondent's self-reported political leanings from "Left"
                to "Right".
            ClinLR - Respondents impression of Bill Clinton's political
                leanings from "Left" to "Right".
            DoleLR  - Respondents impression of Bob Dole's political leanings
                from "Left" to "Right".
            logpopul - log(popul + .1)
```

## 1. Основные свойства DataFrame

DataFrame (`df`) содержит данные о зарегистрированных избирателях в США, включая демографические данные и политические взгляды. Используя `pandas`, выведите первые 5 строк DataFrame, чтобы понять как примерно выглядят данные. Далее ответьте на следующие вопросы:

* Сколько записей в DataFrame?
* Сколько параметров было оценено в исследовании (сколько столбцов)?
* Каков минимальный и максимальный возраст избирателя?
* Сколько раз в неделю в среднем респонденты смотрят новости (округлите до десятых)?
* Проверьте данные на пропущенные значения.

In [59]:
records = len(df)
print(f'Количество записей в df: {records}') # 944

columns = len(df.columns)
print(f'Столбцов: {columns}') # 11

min_age = df['age'].min()
max_age = df['age'].max()
print(f'Минимальный возраст избирателя: {int(min_age)}') # 19
print(f'Максимальный возраст избирателя: {int(max_age)}') # 91

avg_news_watch = df['TVnews'].mean()
print(f'Количество просмотров новостей в неделю: {round(avg_news_watch, 1)}') # 3,7
print()


missing_values = df.isnull().sum()
print(f'Пропущенные значения:\n{missing_values}')

Количество записей в df: 944
Столбцов: 16
Минимальный возраст избирателя: 19
Максимальный возраст избирателя: 91
Количество просмотров новостей в неделю: 3.7

Пропущенные значения:
popul          0
TVnews         0
selfLR         0
ClinLR         0
DoleLR         0
PID            0
age            0
education      0
income         0
vote           0
logpopul       0
party          0
age_group      0
diff           0
diff_ClinLR    0
diff_DoleLR    0
dtype: int64


## 2. Обработка данных

Необходимо настроить датасет для дальнейшего использования. Сделайте следующее:

*   Переименуйте столбец `educ` в `education`.
*   Создайте новый столбец `party` на основе ответов каждого респондента в `PID`. `party` должен быть равен `Democrat`, если респондент выбрал или Strong Democrat, или Weak Democrat. `party` должен быть равен `Republican`, если респондент выбрал Strong или Weak Republican в `PID` и `Independent` во всех остальных случаях.
*   Создайте новый столбец `age_group`, который объединяет респондентов по следующим категориям на основе их возраста `age`: 18-24, 25-34, 35-44, 45-54, 55-64, и 65+.

In [8]:

df.rename(columns={'educ': 'education'}, inplace=True)

def party(pid):
    if pid in [0, 1]:  # Strong Democrat or Weak Democrat
        return 'Democrat'
    elif pid in [5, 6]:  # Strong or Weak Republican
        return 'Republican'
    else:
        return 'Independent'

df['party'] = df['PID'].apply(party)

df['age_group'] = None

for i in range(len(df)):
    age = df.loc[i, 'age']
    
    if 18 <= age <= 24:
        df.loc[i, 'age_group'] = '18-24'
    elif 25 <= age <= 34:
        df.loc[i, 'age_group'] = '25-34'
    elif 35 <= age <= 44:
        df.loc[i, 'age_group'] = '35-44'
    elif 45 <= age <= 54:
        df.loc[i, 'age_group'] = '45-54'
    elif 55 <= age <= 64:
        df.loc[i, 'age_group'] = '55-64'
    elif age >= 65:
        df.loc[i, 'age_group'] = '65+'
    else:
        df.loc[i, 'age_group'] = 'incorrect input'

print(df.head())



   popul  TVnews  selfLR  ClinLR  DoleLR  PID   age  education  income  vote  \
0    0.0     7.0     7.0     1.0     6.0  6.0  36.0        3.0     1.0   1.0   
1  190.0     1.0     3.0     3.0     5.0  1.0  20.0        4.0     1.0   0.0   
2   31.0     7.0     2.0     2.0     6.0  1.0  24.0        6.0     1.0   0.0   
3   83.0     4.0     3.0     4.0     5.0  1.0  28.0        6.0     1.0   0.0   
4  640.0     7.0     5.0     6.0     4.0  0.0  68.0        6.0     1.0   0.0   

   logpopul       party age_group  
0 -2.302585  Republican     35-44  
1  5.247550    Democrat     18-24  
2  3.437208    Democrat     18-24  
3  4.420045    Democrat     25-34  
4  6.461624    Democrat       65+  


## 3. Фильтрация данных

* Используя фильтрацию найдите всех респондентов, кто считал Билла Клинтона умеренным или консерватором (`ClinLR` равно 4 или выше). Сколько респондентов в этом подмножестве?
* Среди всех респондентов, сколько имеют доход ниже $50,000 и хотя бы посещали колледж?

In [55]:
print(df.head())
print()

df['ClinLR'] = pd.to_numeric(df['ClinLR'], errors='coerce')
respondets_ClinLR_ge4 = df[df['ClinLR'] >= 4]

num_of_respondets_ClinLR_ge4 = respondets_ClinLR_ge4.shape[0]
print(f'Респонденты с ClinLR >= 4, число: {num_of_respondets_ClinLR_ge4}')

df['income'] = pd.to_numeric(df['income'], errors='coerce')


college = 1

income_lt_50_college = df[(df['income'] < 50000) & (df['education'] >= college)]


num_of_income_lt_50_college = income_lt_50_college.shape[0]
print(f'Число респондентов с доходом меньше 50.000$, которые посещали колледж (education >= 1): {num_of_income_lt_50_college}')


   popul  TVnews  selfLR  ClinLR  DoleLR  PID   age  education  income  vote  \
0    0.0     7.0     7.0     1.0     6.0  6.0  36.0        3.0     1.0   1.0   
1  190.0     1.0     3.0     3.0     5.0  1.0  20.0        4.0     1.0   0.0   
2   31.0     7.0     2.0     2.0     6.0  1.0  24.0        6.0     1.0   0.0   
3   83.0     4.0     3.0     4.0     5.0  1.0  28.0        6.0     1.0   0.0   
4  640.0     7.0     5.0     6.0     4.0  0.0  68.0        6.0     1.0   0.0   

   logpopul       party age_group  diff  diff_ClinLR  diff_DoleLR  
0 -2.302585  Republican     35-44   6.0          6.0          1.0  
1  5.247550    Democrat     18-24   2.0          0.0          2.0  
2  3.437208    Democrat     18-24   4.0          0.0          4.0  
3  4.420045    Democrat     25-34   2.0          1.0          2.0  
4  6.461624    Democrat       65+   1.0          1.0          1.0  

Респонденты с ClinLR >= 4, число: 282
Число респондентов с доходом меньше 50.000$, которые посещали колледж (e

## 4. Вычисления на основе данных

Для каждого из следующих сочетаний выберите группу, которая более вероятно проголосует за Билла Клинтона. Вы можете это вычислить используя долю каждой группы, которая намерена голосовать за Клинтона (`vote`). Какое сочетание имеет наименьшую разницу, а какое наибольшую?


*   Демократы или республиканцы
*   Люди младше 44 или люди старше 44 лет и старше
*   Люди, которые смотрят новости по меньшей мере 6 дней в неделю, или, которые смотрят менее 3 дней в неделю
*   Люди живущие там, где население меньше чем у среднего респондента, или там, где равно и больше


In [32]:
df['vote'] = pd.to_numeric(df['vote'], errors='coerce')

party_vote = df.groupby('party')['vote'].mean()
democrat_vote_number = party_vote.get('Democrat', 0)
republican_vote_number = party_vote.get('Republican', 0)
party_difference = abs(democrat_vote_number - republican_vote_number)

print(f'1. Демократы: {democrat_vote_number:.2f}, \
      республиканцы: {republican_vote_number:.2f}, \
    разница: {party_difference:.2f}')


age_group_vote = df.groupby(df['age'] < 44)['vote'].mean()
young_vote_number = age_group_vote.get(True, 0)
old_vote_number = age_group_vote.get(False, 0)
age_difference = abs(young_vote_number - old_vote_number)

print(f"2. Младше 44: {young_vote_number:.2f}, \
      старше 44: {old_vote_number:.2f}, \
      разница: {age_difference:.2f}")


tv_news_group_vote = df.groupby((df['TVnews'] >= 6) | (df['TVnews'] < 3))['vote'].mean()
frequent_news_watchers_vote_number = tv_news_group_vote.get(True, 0)
infrequent_news_watchers_vote_number = tv_news_group_vote.get(False, 0)
tv_news_difference = abs(frequent_news_watchers_vote_number - infrequent_news_watchers_vote_number)

print(f"3. Смотрят новости 6 >= дней в неделю: {frequent_news_watchers_vote_number:.2f}, \
      Смотрят новости 3 < дней в неделю: {infrequent_news_watchers_vote_number:.2f}, \
      разница: {tv_news_difference:.2f}")


mean_popul = df['popul'].mean()
popul_group_vote = df.groupby(df['popul'] < mean_popul)['vote'].mean()
less_populated_vote_number = popul_group_vote.get(True, 0)
more_populated_vote_number = popul_group_vote.get(False, 0)
popul_difference = abs(less_populated_vote_number - more_populated_vote_number)

print(f"4. Население меньше, чем у среднего респондента: {less_populated_vote_number:.2f}, \
      больше или равно: {more_populated_vote_number:.2f}, \
      разница: {popul_difference:.2f}")


differences = {
    "party": party_difference,
    "age": age_difference,
    "tv_news": tv_news_difference,
    "popul": popul_difference
}

min_difference = min(differences, key=differences.get)
max_difference = max(differences, key=differences.get)

print()
print(f"Наименьшая разница: {min_difference}")
print(f"Наибольшая разница: {max_difference}")


1. Демократы: 0.04,       республиканцы: 0.90,     разница: 0.86
2. Младше 44: 0.41,       старше 44: 0.43,       разница: 0.02
3. Смотрят новости 6 >= дней в неделю: 0.43,       Смотрят новости 3 < дней в неделю: 0.37,       разница: 0.07
4. Население меньше, чем у среднего респондента: 0.44,       больше или равно: 0.28,       разница: 0.16

Наименьшая разница: age
Наибольшая разница: party


## 5. Группировка данных

Используя метод `groupby()` объедините респондентов по `age_group`. Какая возрастная группа сама консервативная? Какая меньше всех смотрит новости?

Далее, рассчитайте 5-процентильные группы в зависимости от дохода. Сгруппируйте набор данных по этим процентилям. Какая группа доходов является наиболее либеральной? Какая самой консервативной? Старейшей? Наиболее образованной?

In [53]:
age_group = df.groupby('age_group').agg({
    'ClinLR': 'mean',
    'TVnews': 'mean'
})

conservative_group = age_group['ClinLR'].idxmax()
least_news_watching_group = age_group['TVnews'].idxmin()

print(f"Самая консервативная возврастная группа: {conservative_group}; \
    группа, смотрящая новости меньше всего{least_news_watching_group}")

# 5-й процентиль — это значение, ниже которого находятся 5% данных.

income_percentiles = df['income'].quantile([0, 0.05, 0.25, 0.5, 0.75, 0.95, 1]).unique()
income_percentiles.sort()
unique_income_percentiles = pd.unique(income_percentiles)
income_labels = ['0-5%', '5-25%', '25-50%', '50-75%', '75-95%', '95-100%']

if len(unique_income_percentiles) - 1 != len(income_labels):
    # Если нет, обновляем метки, чтобы соответствовать количеству интервалов
    income_labels = [f'{int(100*i/len(unique_income_percentiles))}-{int(100*(i+1)/len(unique_income_percentiles))}%' for i in range(len(unique_income_percentiles) - 1)]

income_groups = pd.cut(df['income'], bins=unique_income_percentiles, labels=income_labels, duplicates='drop')


income_group_stats = df.groupby(income_groups).agg({
    'ClinLR': 'mean',
    'age': 'mean',
    'education': 'mean'
})

# Группы по доходу:
most_liberal = income_group_stats['ClinLR'].idxmin()
most_conservative = income_group_stats['ClinLR'].idxmax()
oldest = income_group_stats['age'].idxmax()
most_educated = income_group_stats['education'].idxmax()

print()
print(f"Группы. Наиболее либеральная: {most_liberal}, \
    самая консервативная: {most_conservative}, \
    старейшая группа: {oldest}, \
    наиболее образованная: {most_educated}")


Самая консервативная возврастная группа: 18-24;     группа, смотрящая новости меньше всего18-24

Группы. Наиболее либеральная: 66-83%,     самая консервативная: 0-16%,     старейшая группа: 16-33%,     наиболее образованная: 66-83%


C:\Users\user\AppData\Local\Temp\ipykernel_1636\3769146446.py:26: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  income_group_stats = df.groupby(income_groups).agg({


## 6. Голосование вне логики

Вам интересно узнать больше о респондентах, чьи политические взгляды сильно отличаются от кандидата, за которого они планируют голосовать. Используя `selfLR`, `vote`, `ClinLR`, и `DoleLR`, ответьте на следующие вопросы:

*   Какая наибольшая известная разница между политическими взглядами респондента и его кандидата?
*   Сколько респондентов показали такую разницу?
*   Сделайте отдельный DataFrame `sway`, который включает только избирателей, которые демонстрируют разницу более $|3|$.
*   В `sway`, больше респондентов голосующих за более либерального или более консервативного кандидата, чем они сами?
*   В `sway`, какой кандидат наиболее популярен?

In [52]:
df['diff_ClinLR'] = abs(df['selfLR'] - df['ClinLR'])
df['diff_DoleLR'] = abs(df['selfLR'] - df['DoleLR'])
df['diff'] = df[['diff_ClinLR', 'diff_DoleLR']].max(axis=1)

max_diff = df['diff'].max()
num_respondents_max_diff = (df['diff'] == max_diff).sum()

print(f"Наибольшая разница между политическими взглядами респондента и его кандидатом: {max_diff}")
print(f"Количество респондентов с такой разницей: {num_respondents_max_diff}")

# 3. Создать DataFrame sway, включающий только респондентов с разницей более 3 (по модулю)
sway = df[df['diff'] > 3]

# 4. В sway определить, больше ли респондентов голосуют за более либерального или более консервативного кандидата, чем они сами

liberal_votes = sway[sway['ClinLR'] < sway['selfLR']]['vote'].count()
conservative_votes = sway[sway['ClinLR'] > sway['selfLR']]['vote'].count()

if liberal_votes > conservative_votes:
    more_votes = 'liberal'
else:
    more_votes = 'conservative'

# 5. Найти наиболее популярного кандидата в sway
most_popular_candidate = sway['vote'].mode().iloc[0]

# Вывод результатов

print(f"Больше респондентов голосуют за более {more_votes} кандидата, чем они сами.")
print(f"Наиболее популярный кандидат в sway: {'Clinton' if most_popular_candidate == 1 else 'Dole'}")

Наибольшая разница между политическими взглядами респондента и его кандидатом: 6.0
Количество респондентов с такой разницей: 21
Больше респондентов голосуют за более liberal кандидата, чем они сами.
Наиболее популярный кандидат в sway: Clinton
